In [1]:
!python --version
!which python

Python 3.12.8
/public/workspace/ryrl/venvs/versions/3.12.8/torch/bin/python


In [2]:
from datasets import load_dataset

dataset = load_dataset('yelp_review_full')

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/299M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/23.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [63]:
dataset['train'][100]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-cased')

def tokenize_function(example):
    return tokenizer(example['text'], padding='max_length', truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [15]:
train_datasets = tokenized_datasets['train'].shuffle(seed=42).select(range(1000))
test_datasets = tokenized_datasets['test'].shuffle(seed=42).select(range(1000))

In [23]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5, torch_dtype="auto")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [25]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [26]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [27]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")

In [28]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [29]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [30]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.096226,0.514000
2,No log,1.011690,0.559000
3,No log,0.990456,0.595000


TrainOutput(global_step=375, training_loss=1.0159188639322916, metrics={'train_runtime': 1791.4007, 'train_samples_per_second': 1.675, 'train_steps_per_second': 0.209, 'total_flos': 789354427392000.0, 'train_loss': 1.0159188639322916, 'epoch': 3.0})

In [43]:
np.unique(tokenized_datasets['train']['label'])

array([0, 1, 2, 3, 4])

In [44]:
tokenized_datasets = tokenized_datasets.remove_columns(['text'])
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [ ]:
tokenized_datasets = tokenized_datasets.rename_column('label' 'labels')
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [48]:
tokenized_datasets.set_format('torch')
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [49]:
small_train_dataset = tokenized_datasets['train'].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets['test'].shuffle(seed=42).select(range(1000))

In [50]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(dataset=small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(dataset=small_eval_dataset, batch_size=8)

In [52]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('google-bert/bert-base-cased', num_labels= 5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [53]:
from torch.optim import Adam

optimizer = Adam(model.parameters(), lr=5e-5)

In [54]:
from transformers import get_scheduler

numEpochs = 3
numTrainSteps = numEpochs * len(train_dataloader)
lr_scheduler = get_scheduler(name='linear', optimizer=optimizer, num_warmup_steps=0, num_training_steps=numTrainSteps)

In [57]:
numTrainSteps, lr_scheduler

(375, <torch.optim.lr_scheduler.LambdaLR at 0x7fd4a4f4f920>)

In [58]:
import torch 
from accelerate.test_utils.testing import get_backend

device, _, _ = get_backend()
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [61]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(numTrainSteps))
model.train()

for epoch in range(numEpochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/375 [00:00<?, ?it/s]

In [65]:
import evaluate

metric = evaluate.load('accuracy')
model.eval()

for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}

    with torch.no_grad():
        outputs = model(**batch)
    
    logits = outputs.logits
    predictions = torch.argmax(logits, axis=-1)
    metric.add_batch(predictions=predictions, references=batch['labels'])

metric.compute()

{'accuracy': 0.603}

## Steps of Fine-tune pretrained model:

- PyTorch Trainer:
  - Preprocess Dataset to Pytorch `Data` Objec.
  - Dataset to Tokenizer by `AutoTokenizer`
  - Model selection
  - Training Arguments `TrainingArguments`
  - Evaluation function:
    ```python
    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return metric.compute(predictions=predictions, references=labels)
    ```
  - Trainer: `transformers.Trainer`, `trainer.train()`
  
- PyTorch Native
  - Preprocess Data to fit the requirement of models and tokenizer
  - Transform the Data by `DataLoader`
  - Model Selection
  - Parameters setting
    ```python
    from transformers import get_scheduler
    from torch.optim import Adam
    import torch
    from accelerate.test_utils.testing import get_backend

    numEpochs = 3
    numTrainSteps = numEpochs * len(train_dataloader)
    lr_scheduler = get_scheduler(name='linear', optimizer=optimizer, num_warmup_steps=0, num_training_steps=numTrainSteps)

    
    optimizer = Adam(model.parameters(), lr=5e-5)
    device, _, _ = get_backend()
    model.to(device)
    ```
  - Train
    ```python
    from tqdm.auto import tqdm

    progress_bar = tqdm(range(numTrainSteps))
    model.train()

    for epoch in range(numEpochs):
        for batch in train_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)
    ```
  - Evalutation
    ```python
    import evaluate

    metric = evaluate.load('accuracy')
    model.eval()

    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}

        with torch.no_grad():
            outputs = model(**batch)
        
        logits = outputs.logits
        predictions = torch.argmax(logits, axis=-1)
        metric.add_batch(predictions=predictions, references=batch['labels'])

    metric.compute()
    ```